# Chapter 4: Simulating and Analyzing MA(1) Time Series

## Simulate MA(1) Time Series

You will simulate and plot a few MA(1) time series, each with a different parameter, using the `arima_process` module in `statsmodels`.

As in the last chapter, when inputting the coefficients, you must include the zero-lag coefficient of 1. Unlike the AR models, the sign of the MA coefficients is what we expect. For example, for an MA(1) process with θ = 0.9, the array representing the MA parameters would be `ma = np.array([1, -0.9])`.


In [ ]:

# Import necessary modules
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_process import ArmaProcess

# Plot 1: MA parameter = -0.9
plt.figure(figsize=(10,6))

plt.subplot(2,1,1)
ar1 = np.array([1])
ma1 = np.array([1, -0.9])
MA_object1 = ArmaProcess(ar1, ma1)
simulated_data_1 = MA_object1.generate_sample(nsample=1000)
plt.plot(simulated_data_1)
plt.title('Simulated MA(1) Process with θ = -0.9')

# Plot 2: MA parameter = +0.9
plt.subplot(2,1,2)
ar2 = np.array([1])
ma2 = np.array([1, 0.9])
MA_object2 = ArmaProcess(ar2, ma2)
simulated_data_2 = MA_object2.generate_sample(nsample=1000)
plt.plot(simulated_data_2)
plt.title('Simulated MA(1) Process with θ = 0.9')

plt.tight_layout()
plt.show()


## Compute the ACF for Several MA Time Series

Unlike an AR(1), an MA(1) model has no autocorrelation beyond lag 1. The lag-1 autocorrelation for an MA(1) model is not θ, but rather `θ / (1 + θ²)`. You will verify these autocorrelation functions for the three time series you generated earlier.
simulated_data_1 is the first simulated time series with an MA parameter of .
Compute the autocorrelation function of simulated_data_1 using the plot_acf function with 20 lags.


In [ ]:

# Import the plot_acf module from statsmodels
from statsmodels.graphics.tsaplots import plot_acf

# Plot 1: MA parameter = -0.9
plot_acf(simulated_data_1, lags=20)
plt.show()

# Plot 2: MA parameter = 0.9
plot_acf(simulated_data_2, lags=20)
plt.show()


## Estimating an MA Model

You will estimate the MA(1) parameter θ of one of the simulated series. Since the parameters are known for a simulated series, it is a good way to understand the estimation routines before applying them to real data.


In [ ]:

# Import the ARMA module from statsmodels
from statsmodels.tsa.arima.model import ARIMA

# Fit an MA(1) model to the first simulated data
mod = ARIMA(simulated_data_1, order=(0,0,1))
res = mod.fit()

# Print out summary information on the fit
print(res.summary())

# Print out the estimate for the constant and for theta
print("When the true theta=-0.9, the estimate of theta (and the constant) are:")
print(res.params)


## Forecasting with MA Model

You will use the MA(1) model to forecast in-sample and out-of-sample data using statsmodels.For the simulated series simulated_data_1 with , you will plot in-sample and out-of-sample forecasts. One big difference you will see between out-of-sample forecasts with an MA(1) model and an AR(1) model is that the MA(1) forecasts more than one period in the future are simply the mean of the sample.



In [ ]:

# Forecast the first MA(1) model
res.plot_predict(start=990, end=1010)
plt.show()


## High Frequency Stock Prices

Higher frequency stock data is well modeled by an MA(1) process, so it's a nice application of the models in this chapter.

The DataFrame intraday contains one day's prices (on September 1, 2017) for Sprint stock (ticker symbol "S") sampled at a frequency of one minute. The stock market is open for 6.5 hours (390 minutes), from 9:30am to 4:00pm.

Before you can analyze the time series data, you will have to clean it up a little, which you will do in this and the next two exercises. When you look at the first few rows (see the IPython Shell), you'll notice several things. First, there are no column headers.The data is not time stamped from 9:30 to 4:00, but rather goes from 0 to 390. And you will notice that the first date is the odd-looking "a1504272600". The number after the "a" is Unix time which is the number of seconds since January 1, 1970. This is how this dataset separates each day of intraday data.

If you look at the data types, you'll notice that the DATE column is an object, which here means a string. You will need to change that to numeric before you can clean up some missing data.

The source of the minute data is Google Finance

In [ ]:

# Import pandas and datetime module
import pandas as pd
import datetime

# Load the intraday data (assume it is already loaded as `intraday`)
# Change the first date to zero
intraday.iloc[0,0] = 0

# Change the column headers to 'DATE' and 'CLOSE'
intraday.columns = ['DATE','CLOSE']

# Convert DATE column to numeric
intraday['DATE'] = pd.to_numeric(intraday['DATE'])

# Make the `DATE` column the new index
intraday = intraday.set_index('DATE')


## More Data Cleaning: Missing Data


When you print out the length of the DataFrame intraday, you will notice that a few rows are missing. There will be missing data if there are no trades in a particular one-minute interval. One way to see which rows are missing is to take the difference of two sets: the full set of every minute and the set of the DataFrame index which contains missing rows. After filling in the missing rows, you can convert the index to time of day and then plot the data.

Stocks trade at discrete one-cent increments (although a small percentage of trades occur in between the one-cent increments) rather than at continuous prices, and when you plot the data you should observe that there are long periods when the stock bounces back and forth over a one cent range. This is sometimes referred to as "bid/ask bounce".


In [ ]:

# Identify missing rows
set_everything = set(range(391))
set_intraday = set(intraday.index)
set_missing = set_everything - set_intraday

print("Missing rows: ", set_missing)

# Fill in missing rows
intraday = intraday.reindex(range(391), method='ffill')

# Change index to time format
intraday.index = pd.date_range(start='2017-09-01 9:30', end='2017-09-01 16:00', freq='1min')

# Plot intraday time series
intraday.plot(grid=True)
plt.show()


## Applying an MA Model

The bouncing of the stock price between bid and ask induces a negative first order autocorrelation, but no autocorrelations at lags higher than 1. You get the same ACF pattern with an MA(1) model. Therefore, you will fit an MA(1) model to the intraday stock data from the last exercise.

The first step is to compute minute-by-minute returns from the prices in intraday, and plot the autocorrelation function. You should observe that the ACF looks like that for an MA(1) process. Then, fit the data to an MA(1), the same way you did for simulated data.


In [ ]:
# Import plot_acf and ARMA modules from statsmodels
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.arima_model import ARMA

# Compute returns from prices
returns = intraday.pct_change().dropna()

# Plot ACF of returns with lags up to 60 minutes
plot_acf(returns, lags=60)
plt.show()

# Fit the returns data to an MA(1) model
mod = ARIMA(returns, order=(0,0,1))
res = mod.fit()
print(res.params)


## Equivalence of AR(1) and MA(infinity)

To better understand the relationship between MA models and AR models, you will simulate an MA model with parameters for 30 lags and show that it has the same ACF as an AR(1) model.


In [ ]:

# Simulate an MA(30) model
ma = np.array([.8**i for i in range(30)])
ar = np.array([1])
AR_object = ArmaProcess(ar, ma)
simulated_data = AR_object.generate_sample(nsample=5000)

# Plot the ACF
plot_acf(simulated_data, lags=30)
plt.show()
